In [ ]:
#Pkg.add("PyCall")

In [13]:

@pyimport larlib

XgeModule::init_started
Creating shared GLCanvas...
shared GLCanvas created
XgeModule::init_finished
Evaluating fenvs.py..
('...fenvs.py imported in', 0.009031999999999485, 'seconds')


# Visualization by Giulio

In [14]:
function lar2plasm(EV, FE)
    EV = sparse(EV)
    faces = Array{Array{Int64, 1}, 1}()

    for f in 1:size(FE, 1)

        verts = Array{Int64, 1}()
        done = Array{Int64, 1}()
        face = FE[f, :]
        edges = EV[face.nzind, :]

        push!(done, 1)
        vs = edges[1, :].nzind
        if face.nzval[1] < 0
            vs = vs[end:-1:1]
        end
        startv, nextv = vs

        println(startv)
        push!(verts, startv)
        while nextv != startv
            e = setdiff(edges[:, nextv].nzind, done)[1]
            push!(done, e)
            vs = edges[e, :].nzind
            if face.nzval[e] < 0
                vs = vs[end:-1:1]
            end
            curv, nextv = vs
            push!(verts, curv)
        end

        push!(faces, verts)
    end

    "FV = " * string(faces)[15:end]
end

function lar2py(V, EV)
    text = "V = [[0,0]"
    for i in 1:size(V, 1)
        text = text * "," * string(vec(V[i, :]))
    end
    text = text * "]\n"
    
    text = text * "EV = ["
    for i in 1:size(EV, 1)
        if i > 1 text = text * "," end
        text = text * string(EV[i, :].nzind)
    end
    text = text * "]"
    return text
end


function visualize_numbers(set, font_size)

    f = open("tmp.py", "w")

    file_content = "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size))\n"
    print(file_content)
    write(f, file_content)

    close(f)
#     run(`python tmp.py`)
end

function visualize_boundary(set, boundary, font_size)
    f = open("tmp.py", "w")
    file_content =  "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "model = larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size)\n" *
        lar2py(set[1], set[2][boundary.nzind, :]) * "\n" *
        "overmodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(STRUCT([model, COLOR(RED)(overmodel)]))"
    print(file_content)
    write(f, file_content)

    close(f)
    run(`python tmp.py`)
end

visualize_boundary (generic function with 1 method)

## My test object

In [15]:
V = [
    1 1
    2 1
    1 2
    2 2
    ]

4×2 Array{Int64,2}:
 1  1
 2  1
 1  2
 2  2

In [16]:
EV = sparse([
    1 -1 0 0
    0 1 -1 0
    -1 0 1 0
])

3×4 sparse matrix with 6 Int64 nonzero entries:
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1

In [17]:
FE = sparse([
        1 1 -1
        -1 -1 1
])

2×3 sparse matrix with 6 Int64 nonzero entries:
	[1, 1]  =  1
	[2, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[1, 3]  =  -1
	[2, 3]  =  1

In [18]:
set=[V, EV, FE]

3-element Array{AbstractArray{Int64,2},1}:
 [1 1; 2 1; 1 2; 2 2]                                                                   
 
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
 
	[1, 1]  =  1
	[2, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[1, 3]  =  -1
	[2, 3]  =  1

## lar2py()

In [19]:
lar2py(set[1], set[2]) 

"V = [[0,0],[1,1],[2,1],[1,2],[2,2]]\nEV = [[1,2],[2,3],[1,3]]"

## lar2plasm()

In [20]:
lar2plasm(set[2], set[3])

1
2


"FV = [[1,2,3],[2,1,3]]"

In [21]:
visualize_numbers([V, EV, FE],5)

1
2
from larlib import *
V = [[0,0],[1,1],[2,1],[1,2],[2,2]]
EV = [[1,2],[2,3],[1,3]]
FV = [[1,2,3],[2,1,3]]
VV = AA(LIST)(range(len(V)))
submodel = STRUCT(MKPOLS((V,EV)))
VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,5))


## Run python

In [22]:
println(pwd())
unshift!(PyVector(pyimport("sys")["path"]), "")

/home/mjirik/projects/lario3d.jl/notebooks


PyObject ['', '/home/mjirik/.local/lib/python2.7/site-packages/mahotas-1.4.1-py2.7-linux-x86_64.egg', '/home/mjirik/miniconda2/envs/jl_py27/lib/python27.zip', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7/plat-linux2', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7/lib-tk', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7/lib-old', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7/lib-dynload', '/home/mjirik/.local/lib/python2.7/site-packages', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7/site-packages', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7/site-packages/poly2tri-0.3.3+dcpatch-py2.7-linux-x86_64.egg', '/home/mjirik/miniconda2/envs/jl_py27/lib/python2.7/site-packages/larlib-0.3.2-py2.7.egg']

In [23]:
using PyCall
@pyimport tmp

centre
centre
centre
Building batches from Hpc....
...done in 0 msec
Optimizing the octree....
   Number of input batches 73
   total number vertices    104
   Number of output batches 52
   Batch vertex media       2
...done in 0 msec
Building octree from 52 batches....
Scene number of nodes of the octree 64
Scene max depth                     4
Scene number of batches             52
...done in 0 msec
